### 데이터 전처리

In [8]:
import pandas as pd
from __future__ import annotations

# ===== Standard Library =====
import os
import re
import math
import argparse
from typing import List, Tuple, Dict, Iterable
from collections import Counter, defaultdict
from itertools import combinations

# ===== Third-Party =====
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.optimize import linear_sum_assignment

# Text / Features
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Dimensionality Reduction
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.manifold import TSNE

# Clustering
from sklearn.cluster import KMeans

# Metrics
from sklearn.metrics import (
    f1_score,
    adjusted_rand_score,
    homogeneity_score,
    confusion_matrix,
    accuracy_score,
    recall_score,
    classification_report,
)

# Preprocessing
from sklearn.preprocessing import normalize

from dotenv import load_dotenv
load_dotenv()

True

In [9]:
df = pd.read_csv('/home/ys0660/2507Sub/KARD/fortopicmodeling_v2.csv')

In [10]:
print(df.head())

   Unnamed: 0  index1                                            context  \
0           0   16191                           혹시나 했는데 역시 헌금때문에 하는게 맞군요   
1           1   24855                              배추앓이님 얼마나 당황스럽겠읍니까 ㅋㅋ   
2           2   20864                      작년 연말에 기독교랑 이슬람이랑 막 총질해대지 않았나   
3           3   11884  꼬우면 니들도 짱깨 짱꼴라 짱퀴벌레 우한폐렴 소년단 만들어라 ㅋ 너넨 지옥의 짱깨 ...   
4           4   10915  근데 저새끼는 저러고 다시 원상복귀함 속좁고 옹졸한게 맨날 지 여친한테도 뱀눈까리뜨...   

    Label   혐오 대상 혐오대상(세부)  
0      정상     NaN      NaN  
1      정상     NaN      NaN  
2      정상     NaN      NaN  
3  명백한 혐오  ['인종']   ['중국']  
4      욕설     NaN      NaN  


In [11]:
len(df)

25933

In [12]:
import ast
import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

def parse_multi_labels(x):
    if pd.isna(x):
        return []
    if isinstance(x, list):
        return x
    if isinstance(x, str):
        try:
            val = ast.literal_eval(x) 
            val = [str(v).strip() for v in val]
            val = [v for v in val if v]
            return val
        except Exception:
            return [x.strip()]
    return [str(x).strip()]

# 1) 파싱
df["label"] = df["혐오 대상"].apply(parse_multi_labels)

# 2) 멀티-원핫 변환
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(df["label"])   # shape: (N, C)

# 3) DataFrame으로 붙이기 (boolean 또는 0/1)
label_cols = mlb.classes_.tolist()
df_onehot = pd.DataFrame(Y, columns=label_cols, index=df.index).astype(np.uint8)

# 4) 원본에 합치기
df = pd.concat([df.drop(columns=["혐오 대상"]), df_onehot], axis=1)

print("라벨 컬럼들:", label_cols)
print(df.head())


라벨 컬럼들: ['기타', '성소수자', '연령', '인종', '장애인', '정치', '젠더', '종교', '지역']
   Unnamed: 0  index1                                            context  \
0           0   16191                           혹시나 했는데 역시 헌금때문에 하는게 맞군요   
1           1   24855                              배추앓이님 얼마나 당황스럽겠읍니까 ㅋㅋ   
2           2   20864                      작년 연말에 기독교랑 이슬람이랑 막 총질해대지 않았나   
3           3   11884  꼬우면 니들도 짱깨 짱꼴라 짱퀴벌레 우한폐렴 소년단 만들어라 ㅋ 너넨 지옥의 짱깨 ...   
4           4   10915  근데 저새끼는 저러고 다시 원상복귀함 속좁고 옹졸한게 맨날 지 여친한테도 뱀눈까리뜨...   

    Label 혐오대상(세부) label  기타  성소수자  연령  인종  장애인  정치  젠더  종교  지역  
0      정상      NaN    []   0     0   0   0    0   0   0   0   0  
1      정상      NaN    []   0     0   0   0    0   0   0   0   0  
2      정상      NaN    []   0     0   0   0    0   0   0   0   0  
3  명백한 혐오   ['중국']  [인종]   0     0   0   1    0   0   0   0   0  
4      욕설      NaN    []   0     0   0   0    0   0   0   0   0  


In [12]:
len(df)

25933

### openAI

In [13]:
# api call
n_clusters = 9
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
all_vecs = []
batch_size = 128  # 배치 크기 조정 가능
model_name = "text-embedding-3-small"
import tiktoken

enc = tiktoken.encoding_for_model("text-embedding-3-small")

MAX_TOKENS = 8192

def truncate_text(text, max_tokens=MAX_TOKENS):
    tokens = enc.encode(text)
    if len(tokens) > max_tokens:
        tokens = tokens[:max_tokens]
    return enc.decode(tokens)

print("[2] Call API for Embeddings...")

def get_embed_dim(model_name: str) -> int:
    name = model_name.lower()
    if "text-embedding-3-large" in name:
        return 3072
    # small/ada-002 등 기본 1536
    return 1536


def embed_texts_and_align_labels(
    texts: List[str],
    labels: List[int],
    model: str = "text-embedding-3-small"
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, Dict[str, List[int]]]:
    assert len(texts) == len(labels), f"len(texts)={len(texts)} vs len(labels)={len(labels)}"
    N = len(texts)
    D = get_embed_dim(model)

    X = np.zeros((N, D), dtype=np.float32)
    mask_nonzero = np.zeros(N, dtype=bool)

    ok_idx: List[int] = []
    empty_idx: List[int] = []
    fail_idx: List[int] = []

    for idx, raw in enumerate(tqdm(texts, desc="Embedding", total=N)):
        text = truncate_text(raw)
        if not text:
            empty_idx.append(idx)
            continue
        try:
            resp = client.embeddings.create(model=model, input=text)
            vec = np.asarray(resp.data[0].embedding, dtype=np.float32)

            if vec.shape[0] != D:
                print(f"[warn] idx={idx} dim change: {vec.shape[0]} != {D}. Adjusting matrix.")
                newD = vec.shape[0]
                if newD > D:
                    pad = np.zeros((N, newD - D), dtype=np.float32)
                    X = np.hstack([X, pad])  # 오른쪽에 0 패딩
                else:
                    X = X[:, :newD]
                D = newD

            X[idx] = vec
            mask_nonzero[idx] = True
            ok_idx.append(idx)
        except Exception as e:
            print(f"[warn] idx={idx} embedding failed: {e!r}")
            fail_idx.append(idx)

    # 요약 로그
    print("\n=== Embedding Summary ===")
    print(f"Total inputs   : {N}")
    print(f"OK             : {len(ok_idx)}")
    print(f"Empty texts    : {len(empty_idx)}")
    print(f"API failures   : {len(fail_idx)}")
    print(f"Kept (non-zero): {mask_nonzero.sum()} | Dropped: {(~mask_nonzero).sum()}")
    if empty_idx[:5]:
        print(f"First empty idx: {empty_idx[:5]}")
    if fail_idx[:5]:
        print(f"First fail idx : {fail_idx[:5]}")

    # 같은 마스크로 라벨/텍스트 필터링
    labels_arr = np.asarray(labels)
    texts_arr  = np.asarray(texts, dtype=object)
    labels_clean = labels_arr[mask_nonzero]
    texts_clean  = texts_arr[mask_nonzero]

    logs = {"ok": ok_idx, "empty": empty_idx, "fail": fail_idx, "dim": D}
    return X, mask_nonzero, labels_clean, texts_clean, logs

[2] Call API for Embeddings...


In [14]:
from pathlib import Path
import numpy as np

def save_clean_pack(save_path: str, X_clean: np.ndarray,
                    labels_clean: np.ndarray, texts_clean: np.ndarray):
    Path(save_path).parent.mkdir(parents=True, exist_ok=True)
    np.savez(save_path, X=X_clean, labels=labels_clean, texts=texts_clean)
    print(f"[saved] {save_path} | X={X_clean.shape}, labels={labels_clean.shape}, texts={texts_clean.shape}")

# 예시: df_clean에 다음 컬럼이 원핫 라벨이라고 가정
label_cols = ['기타','성소수자','연령','인종','장애인','정치','젠더','종교','지역']  # 실제 보유 컬럼명으로 교체

texts = df["context"].astype(str).tolist()   # 텍스트는 리스트로
y_true = df[label_cols].astype("uint8").values  # (N, C) 멀티-원핫 행렬

# 실제 실행
X, mask_nonzero, labels_clean, texts_clean, logs = embed_texts_and_align_labels(
    texts, y_true, model="text-embedding-3-small"
)
X_clean = X[mask_nonzero]  # 여기서만 마스크 적용

print("Before/After:", X.shape, "->", X_clean.shape)

# 저장
save_path = "/home/ys0660/2507Sub/KARD/data.npz"
save_clean_pack(save_path, X_clean, labels_clean, texts_clean)

Embedding: 100%|██████████| 25933/25933 [2:45:53<00:00,  2.61it/s]  


=== Embedding Summary ===
Total inputs   : 25933
OK             : 25933
Empty texts    : 0
API failures   : 0
Kept (non-zero): 25933 | Dropped: 0
Before/After: (25933, 1536) -> (25933, 1536)
[saved] /home/ys0660/2507Sub/KARD/data.npz | X=(25933, 1536), labels=(25933, 9), texts=(25933,)


In [ ]:
save_path = "/home/ys0660/2507Sub/KARD/data.npz"
save_clean_pack(save_path, X, labels_clean, texts_clean)

### npz load

In [13]:
save_path = "/home/ys0660/2507Sub/KARD/data.npz"

# 불러오기
data = np.load(save_path, allow_pickle=True)
X, y_true, texts = data["X"], data["labels"], data["texts"]
print("Loaded:", X.shape, y_true.shape, texts.shape)

Loaded: (25933, 1536) (25933, 9) (25933,)


### case1

In [14]:
target_labels = ["명백한 혐오", "맥락적 혐오", "모호한 혐오"]
case1_df = df[df["Label"].isin(target_labels)].copy()

In [15]:
import re
import numpy as np
import pandas as pd

def norm(s: str) -> str:
    s = str(s)
    s = re.sub(r"\s+", " ", s)   # 여러 공백 → 1칸
    return s.strip().lower()

# 1) 정규화
case1_df_unique = case1_df.drop_duplicates(subset=["context"]).copy()
case1_norm = case1_df_unique["context"].map(norm).values
texts_norm = np.array([norm(t) for t in texts])

# 2) texts의 "값 → 첫 인덱스" 사전 만들기 (대표 인덱스)
first_idx = {}
for i, v in enumerate(texts_norm):
    if v not in first_idx:
        first_idx[v] = i

# 3) case1 쪽 고유값에 대해 교집합만 첫 인덱스 뽑기 (값당 1개)
targets = pd.unique(case1_norm)
case1_first_idx = [first_idx[v] for v in targets if v in first_idx]

print("case1 고유값 수:", len(targets))
print("교집합 수(= 뽑힌 인덱스 수):", len(case1_first_idx))
print("예시 인덱스 몇 개:", case1_first_idx[:10])

# texts를 DF로 만들고 정규화/대표 인덱스 확보
texts_df = pd.DataFrame({"context": texts})
texts_df["key"] = texts_df["context"].map(norm)
texts_df["orig_idx"] = np.arange(len(texts_df))

# 같은 key가 여러 번 있으면 첫 번째(대표)만 유지
texts_df_first = texts_df.drop_duplicates(subset=["key"], keep="first")

# case1도 정규화/고유화
case1_df_u = case1_df.drop_duplicates(subset=["context"]).copy()
case1_df_u["key"] = case1_df_u["context"].map(norm)

# 1:1 merge → texts의 대표 인덱스만 들어옴
m = case1_df_u.merge(texts_df_first[["key","orig_idx"]], on="key", how="inner")

case1_first_idx = m["orig_idx"].to_numpy()
print("매칭된 대표 인덱스 수:", len(case1_first_idx))


case1 고유값 수: 10432
교집합 수(= 뽑힌 인덱스 수): 10432
예시 인덱스 몇 개: [3, 5, 9, 12, 14, 15, 17, 21, 25, 30]
매칭된 대표 인덱스 수: 10432


### case2

In [16]:
target_labels = ["정상", "맥락적 정상"]
case2_df = df[df["Label"].isin(target_labels)].copy()

In [17]:
case2_df_unique = case2_df.drop_duplicates(subset=["context"]).copy()
case2_norm = case2_df_unique["context"].map(norm).values
texts_norm = np.array([norm(t) for t in texts])

# 2) texts의 "값 → 첫 인덱스" 사전 만들기 (대표 인덱스)
first_idx = {}
for i, v in enumerate(texts_norm):
    if v not in first_idx:
        first_idx[v] = i

# 3) case2 쪽 고유값에 대해 교집합만 첫 인덱스 뽑기 (값당 1개)
targets = pd.unique(case2_norm)
case2_first_idx = [first_idx[v] for v in targets if v in first_idx]

print("case2 고유값 수:", len(targets))
print("교집합 수(= 뽑힌 인덱스 수):", len(case2_first_idx))
print("예시 인덱스 몇 개:", case2_first_idx[:10])

# texts를 DF로 만들고 정규화/대표 인덱스 확보
texts_df = pd.DataFrame({"context": texts})
texts_df["key"] = texts_df["context"].map(norm)
texts_df["orig_idx"] = np.arange(len(texts_df))

# 같은 key가 여러 번 있으면 첫 번째(대표)만 유지
texts_df_first = texts_df.drop_duplicates(subset=["key"], keep="first")

# case1도 정규화/고유화
case2_df_u = case2_df.drop_duplicates(subset=["context"]).copy()
case2_df_u["key"] = case2_df_u["context"].map(norm)

# 1:1 merge → texts의 대표 인덱스만 들어옴
m = case2_df_u.merge(texts_df_first[["key","orig_idx"]], on="key", how="inner")

case2_first_idx = m["orig_idx"].to_numpy()
print("매칭된 대표 인덱스 수:", len(case2_first_idx))


case2 고유값 수: 10222
교집합 수(= 뽑힌 인덱스 수): 10222
예시 인덱스 몇 개: [0, 1, 2, 6, 7, 10, 11, 13, 16, 19]
매칭된 대표 인덱스 수: 10222


In [18]:
## 한국어 데이터셋용 
import re, math
from collections import Counter, defaultdict
from typing import List, Dict, Iterable

def build_topics_from_clusters(
    texts: List[str],
    labels_pred: List[int],
    topn: int = 10,
    min_word_len: int = 2,
    include_noise: bool = False,         # -1 라벨 포함 여부
    stopwords: Iterable[str] = None,
    tokenizer_preference: str = "auto",  # "auto" | "kiwi" | "okt" | "regex"
) -> List[List[str]]:
    """
    한국어 데이터셋용 토픽 단어 추출 함수 (c-TF-IDF 기반)
    - 형태소 분석기(kiwi/okt)가 있으면 우선 사용, 없으면 정규식으로 대체
    - 클러스터별 상위 c-TF-IDF 단어를 반환 (index == cluster_id)
    - 존재하지 않는 cluster_id 위치는 []로 채움
    """

    # ----- 0) 준비: stopwords -----
    default_stop = {
        "것","수","등","및","더","그리고","그러나","하지만","이","그","저","요",
        "합니다","하였다","했다","하는","하게","하며","에서","으로","에게","까지",
        "부터","보다","라고","하면","되다","하다","같다","때문",
        "중","후","전","또한","그리고","입니다",
        "새끼", "진짜", "존나", "시발", "사람", "그냥", "생각"
    }
    stopset = set(default_stop)
    if stopwords:
        stopset |= set(stopwords)

    # ----- 1) 토크나이저 선택 -----
    tok_mode = None
    kiwi = None
    okt = None

    def _try_import_kiwi():
        nonlocal kiwi
        try:
            from kiwipiepy import Kiwi
            kiwi = Kiwi()
            return True
        except Exception:
            return False

    def _try_import_okt():
        nonlocal okt
        try:
            from konlpy.tag import Okt
            okt = Okt()
            return True
        except Exception:
            return False

    if tokenizer_preference == "kiwi":
        tok_mode = "kiwi" if _try_import_kiwi() else "regex"
    elif tokenizer_preference == "okt":
        tok_mode = "okt" if _try_import_okt() else "regex"
    elif tokenizer_preference == "regex":
        tok_mode = "regex"
    else:  # "auto"
        if _try_import_kiwi():
            tok_mode = "kiwi"
        elif _try_import_okt():
            tok_mode = "okt"
        else:
            tok_mode = "regex"

    # ----- 2) 토큰화 함수 -----
    hangul_alnum = re.compile(r"[가-힣A-Za-z0-9]+")
    digits_only  = re.compile(r"^\d+$")

    def tokenize(doc: str) -> List[str]:
        doc = (doc or "").strip()
        if not doc:
            return []

        if tok_mode == "kiwi":
            keep_pos = {"NNG", "NNP", "SL"}  # 필요 시 조정
            toks = []
            for token, pos, _, _ in kiwi.analyze(doc, top_n=1)[0][0]:
                if pos in keep_pos and len(token) >= min_word_len and token not in stopset:
                    toks.append(token)
            return toks

        if tok_mode == "okt":
            nouns = okt.nouns(doc)
            return [w for w in nouns if len(w) >= min_word_len and w not in stopset]

        # regex fallback
        raw = hangul_alnum.findall(doc)
        toks = []
        for w in raw:
            w = w.lower()
            if len(w) < min_word_len:
                continue
            if digits_only.match(w):
                continue
            if w in stopset:
                continue
            toks.append(w)
        return toks

    # ----- 3) 클러스터 집합/크기 계산 -----
    unique_labels = sorted(set(labels_pred))
    if not include_noise and -1 in unique_labels:
        unique_labels.remove(-1)
    if not unique_labels:
        return []

    max_cid = max([lab for lab in unique_labels if lab >= 0] or [0])
    topics: List[List[str]] = [[] for _ in range(max_cid + 1)]

    # ----- 4) 입력 길이 검증 -----
    assert len(texts) == len(labels_pred), "texts와 labels_pred 길이가 다릅니다."

    # ----- 5) cluster_id -> 문서 리스트 수집 -----
    cluster_docs: Dict[int, List[str]] = {}
    for i, lab in enumerate(labels_pred):
        if lab == -1 and not include_noise:
            continue
        if lab < 0:
            continue
        cluster_docs.setdefault(lab, []).append(texts[i])

    if not cluster_docs:
        return topics

    # ----- 6) 각 클러스터 토큰화 -----
    cluster_tokens: Dict[int, List[str]] = {}
    for cid, docs in cluster_docs.items():
        toks = []
        for doc in docs:
            toks.extend(tokenize(doc))
        cluster_tokens[cid] = toks

    # ----- 7) 어휘 사전 구축 -----
    vocab: Dict[str, int] = {}
    for toks in cluster_tokens.values():
        for w in toks:
            if w not in vocab:
                vocab[w] = len(vocab)
    V = len(vocab)
    if V == 0:
        return topics

    inv_vocab = {idx: w for w, idx in vocab.items()}

    # ----- 8) 클러스터-단어 카운트 행렬 (TF) -----
    counts = {cid: [0] * V for cid in cluster_tokens.keys()}
    total_words_per_class = {}
    for cid, toks in cluster_tokens.items():
        arr = counts[cid]
        for w in toks:
            arr[vocab[w]] += 1
        total_words_per_class[cid] = sum(arr)

    # ----- 9) c-TF-IDF 계수 계산 -----
    # tf_t: 모든 클러스터에서의 term 총 빈도
    tf_t = [0] * V
    for arr in counts.values():
        for j in range(V):
            tf_t[j] += arr[j]

    C = len(cluster_tokens)  # 비어있지 않은 클러스터 수
    A = sum(total_words_per_class.values()) / float(C)  # 클래스별 평균 단어 수

    # inv_class_part[j] = log(1 + A / tf_t[j])
    inv_class_part = [0.0] * V
    for j in range(V):
        inv_class_part[j] = math.log(1.0 + (A / tf_t[j])) if tf_t[j] > 0 else 0.0

    # ----- 10) c-TF-IDF 상위 단어 추출 -----
    for cid in range(max_cid + 1):
        if cid not in cluster_tokens:
            topics[cid] = []
            continue

        arr = counts[cid]
        # W_{t,c} = tf_{t,c} * log(1 + A / tf_t)
        scores = [(j, arr[j] * inv_class_part[j]) for j in range(V) if arr[j] > 0]
        if not scores:
            topics[cid] = []
            continue

        scores.sort(key=lambda x: x[1], reverse=True)
        top_items = scores[:topn]
        topics[cid] = [inv_vocab[j] for j, _ in top_items]

    return topics


In [19]:
import json
import numpy as np
import pandas as pd

### K-means++
def cluster_kmeans(X: np.ndarray, n_clusters: int, seed: int = 42) -> np.ndarray:
    km = KMeans(
        n_clusters=n_clusters, 
        init="k-means++", 
        n_init=30,
        random_state=seed  
    )
    labels = km.fit_predict(X)
    return labels

def hungarian_match(labels_true: List[int], labels_pred: List[int]) -> Tuple[Dict[int, int], np.ndarray]:
    """
    예측 클러스터 라벨(labels_pred)을 실제 라벨(labels_true)에 최대 일치하도록 재매핑.
    반환: (매핑 딕셔너리, 재매핑된 예측 라벨 배열)
    """
    y_true = np.asarray(labels_true)
    y_pred = np.asarray(labels_pred)

    # 라벨들을 0..K-1로 압축(불연속 라벨 대비)
    true_ids, y_true_comp = np.unique(y_true, return_inverse=True)
    pred_ids, y_pred_comp = np.unique(y_pred, return_inverse=True)

    K_true = true_ids.size
    K_pred = pred_ids.size
    K = max(K_true, K_pred)

    # 혼동행렬 (사이즈를 같게 맞추기 위해 패딩)
    cm = confusion_matrix(y_true_comp, y_pred_comp, labels=np.arange(max(K_true, K_pred)))
    if cm.shape[0] < K or cm.shape[1] < K:
        pad_r = K - cm.shape[0]
        pad_c = K - cm.shape[1]
        cm = np.pad(cm, ((0, pad_r), (0, pad_c)), mode="constant", constant_values=0)

    # 비용 행렬 = -cm (최대 매칭을 최소 비용으로)
    cost = -cm
    row_ind, col_ind = linear_sum_assignment(cost)

    # pred 라벨(압축 기준) -> true 라벨(원본 기준) 매핑 구성
    mapping_comp = {int(col): int(row) for row, col in zip(row_ind, col_ind)}
    # 압축 라벨을 원래 라벨 값으로 복원
    mapping = {}
    for pred_comp, true_comp in mapping_comp.items():
        if pred_comp < K_pred and true_comp < K_true:
            mapping[int(pred_ids[pred_comp])] = int(true_ids[true_comp])

    # 재매핑 적용
    y_pred_aligned = np.array([mapping.get(lbl, lbl) for lbl in y_pred], dtype=int)
    return mapping, y_pred_aligned


def class_hungarian_metrics(metrics: dict, label_names=None, save_prefix: str | None = None):
    """
    metrics: evaluate_with_hungarian_class(...) 반환 dict
    label_names: {int_label: "name"} 형태 선택 제공
    save_prefix: "runs/exp1" 처럼 주면 CSV/JSON 저장
    """
    # ---- 요약 ----
    acc = metrics.get("accuracy")
    f1w = metrics.get("f1_weighted")
    ari = metrics.get("ari")
    hs  = metrics.get("homogeneity")
    print("=== Summary ===")
    print(f"ACC={acc:.4f} | F1(w)={f1w:.4f} | ARI={ari:.4f} | HS={hs:.4f}")

    # ---- 매핑 표 ----
    mapping = metrics.get("mapping", {})
    if mapping:
        df_map = pd.DataFrame(sorted(mapping.items()), columns=["pred_cluster", "mapped_label"])
        if label_names:
            df_map["mapped_name"] = df_map["mapped_label"].map(lambda x: label_names.get(x, x))
        print("\n=== Hungarian Mapping (pred -> true) ===")
        print(df_map.to_string(index=False))

    # ---- 클래스별 리포트 ----
    report = metrics.get("classification_report", {})
    # classification_report dict에는 'accuracy','macro avg','weighted avg'도 섞여 있으므로 숫자 키만 선택
    per_class = {k: v for k, v in report.items() if k.isdigit()}
    df_cls = pd.DataFrame(per_class).T
    # 타입 정리 및 정렬
    for col in ["precision", "recall", "f1-score", "support"]:
        if col in df_cls:
            df_cls[col] = pd.to_numeric(df_cls[col], errors="coerce")
    df_cls.index = df_cls.index.astype(int)
    if label_names:
        df_cls.insert(0, "name", df_cls.index.map(lambda x: label_names.get(x, x)))
    df_cls = df_cls.sort_values("support", ascending=False)

    print("\n=== Per-Class (sorted by support) ===")
    # 소수점 보기 좋게
    display_cols = [c for c in ["name","precision","recall","f1-score","support"] if c in df_cls.columns]
    print(df_cls[display_cols].round(4).to_string())

    # ---- 최고/최저 리콜 TOP5 ----
    if "recall" in df_cls:
        worst5 = df_cls.nsmallest(5, "recall")
        best5  = df_cls.nlargest(5, "recall")
        print("\n=== Lowest Recall TOP5 ===")
        print(worst5[display_cols].round(4).to_string())
        print("\n=== Highest Recall TOP5 ===")
        print(best5[display_cols].round(4).to_string())

    # ---- 저장 옵션 ----
    if save_prefix:
        # 원본 dict 저장
        with open(f"{save_prefix}_metrics.json", "w", encoding="utf-8") as f:
            json.dump(metrics, f, ensure_ascii=False, indent=2)
        # 표 저장
        df_map.to_csv(f"{save_prefix}_mapping.csv", index=False)
        df_cls.to_csv(f"{save_prefix}_per_class.csv")
        print(f"\n[saved] {save_prefix}_metrics.json / {save_prefix}_mapping.csv / {save_prefix}_per_class.csv")

In [20]:
X_case1 = X[case1_first_idx]
texts_case1 = texts[case1_first_idx]

In [41]:
X_case2 = X[case2_first_idx]
texts_case2 = texts[case2_first_idx]

In [46]:
n_clusters = 30
topn = 30

In [47]:
y_pred2 = cluster_kmeans(X_case2, n_clusters=n_clusters, seed=42)

In [48]:
case2_df_unique["cluster30"] = y_pred2

In [49]:
### 상위 단어 추출
from collections import Counter
import re

topics = build_topics_from_clusters(texts_case2, y_pred2, topn=topn)
for i, topic in enumerate(topics):
    print(f"Topic {i}: {topic}")

Topic 0: ['기온', '광주', '전라도', '전주', '바람', '지역', '대구', '전국', '서울', '부산', '지방', '경상도', '다소', '주차', '해상', '지하철', '동남아', '제주', '축제', '기타', '보통', '기질', '하늘', '사투리', '남해', '양호', '예상', '군산', '전날', '내륙']
Topic 1: ['양형모', '머리', '굴맨', '냄새', '염색', '어그', '느낌', '클릭', '감염', '이미지', '끼리', '얼굴', '보기', '가슴', '제일', '악마', '보틀', '정도', '처리', '오지', '징징', '히든', '똥파리', '자체', '조각', '색상', '콘돔', '접대', '설정', '똥꼬']
Topic 2: ['치카', '이슬람', '클라스', '무슬림', '게이', '영화', '리버풀', '추천', '코스프레', '호프', '클럽', '테러리스트', '선수', '캐릭터', '린더', '엘리엇', '신부', '레알', '발매', '마블', '히어로', '테러', '공개', '에브라', '미코', '레이드', '러시아', '피터', '토씨', '스타']
Topic 3: ['문학', '인간', '과학', '존재', '이해', '논리', '자기', '자체', '사실', '문제', '정의', '그게', '지능', '대해', '앨빈', '정신', '폭력', '지식', '정신병', '하나님', '충격', '의미', '가지', '공부', '창조', '사회', '언어', '가치', '축제', '니체']
Topic 4: ['단어', '소리', '똥꼬', '영어', '좌빨', '표현', '짱깨', '개독', '얘기', '일베', '용어', '게이', '헛소리', '혐오', '자체', '의미', '일베충', '무슨', '사투리', '이해', '발음', '말로', '언어', '남충', '하나', '문자', '정도', '듣기', '이란', '우리']
Topic 5: ['댓글', '기사', 

In [50]:
case2_df_unique.drop(columns=["Unnamed: 0"], inplace=True)


In [52]:
case1_df_unique.head()

,index1,context,Label,혐오대상(세부),label,기타,성소수자,연령,인종,장애인,정치,젠더,종교,지역,cluster10,cluster20,cluster30
3,11884,꼬우면 니들도 짱깨 짱꼴라 짱퀴벌레 우한폐렴 소년단 만들어라 ㅋ 너넨 지옥의 짱깨 ...,명백한 혐오,['중국'],[인종],0,0,0,1,0,0,0,0,0,2,9,23
5,14336,"이슬람의 발생이나 발전단계를 찾아보면, 유대교가 기독교를 무시하고 천주교도 무시하는...",맥락적 혐오,['이슬람교계'],[종교],0,0,0,0,0,0,0,1,0,8,8,17
9,19994,옆에서 쭝얼쭝얼거리면 걍 싫다는듯 시선 피하고 있기만 함 나는 놈들 붙으면 개정색하...,모호한 혐오,['기독교계'],[종교],0,0,0,0,0,0,0,1,0,0,16,2
12,15575,헌재에 진보성향 한명 더 넣고 언중법 유튜브와 유사언론까지 확장하면 차금법 이상의 ...,명백한 혐오,['게이관련'],[성소수자],0,1,0,0,0,0,0,0,0,5,4,1
14,11424,문제를 제기하면 그건 니 문제다 야훼에게 의문을 가지면 믿음이 모자라서 그렇다 개독...,명백한 혐오,['기독교계'],[종교],0,0,0,0,0,0,0,1,0,0,4,9


In [51]:
case2_df_unique.head()

,index1,context,Label,혐오대상(세부),label,기타,성소수자,연령,인종,장애인,정치,젠더,종교,지역,cluster10,cluster20,cluster30
0,16191,혹시나 했는데 역시 헌금때문에 하는게 맞군요,정상,NaN,[],0,0,0,0,0,0,0,0,0,2,11,22
1,24855,배추앓이님 얼마나 당황스럽겠읍니까 ㅋㅋ,정상,NaN,[],0,0,0,0,0,0,0,0,0,7,5,1
2,20864,작년 연말에 기독교랑 이슬람이랑 막 총질해대지 않았나,정상,NaN,[],0,0,0,0,0,0,0,0,0,4,4,27
6,12985,비율 와,정상,NaN,[],0,0,0,0,0,0,0,0,0,3,11,22
7,22660,이짤 줏었는데 ㅋㅋㅋ 수박타령 똥파리타령 암컷타령 표 주는 사람이 이상하지 ㅋㅋ 그...,정상,NaN,[],0,0,0,0,0,0,0,0,0,5,17,26


In [53]:
case2_df_unique.to_csv('/home/ys0660/2507Sub/KARD/data/case2_v2.csv')

In [ ]:
from typing import List, Dict
import numpy as np
from sklearn.metrics import (
    accuracy_score, recall_score, f1_score, adjusted_rand_score,
    homogeneity_score, classification_report, silhouette_score,
    calinski_harabasz_score
)

def compute_npmi(topics, texts, topn=10):
    """
    토픽 coherence (NPMI) 계산용 placeholder 함수.
    실제 구현은 Palmetto, Gensim, OCTIS 등의 라이브러리 사용 권장.
    """
    return 0.0

def compute_td(topics, topn=10):
    """
    Topic Diversity (TD): 전체 토픽의 상위 단어 중 고유 단어 비율.
    topics: List[List[str]] 형태 (각 토픽별 상위 단어 리스트)
    """
    all_words = []
    for t in topics:
        all_words.extend(t[:topn])
    return len(set(all_words)) / len(all_words) if all_words else 0.0

def evaluate_with_hungarian(
    labels_true: List[int], 
    labels_pred: List[int], 
    X: np.ndarray = None, 
    topics: List[List[str]] = None, 
    texts: List[str] = None
) -> Dict:
    """
    labels_true: ground truth labels
    labels_pred: predicted labels
    X: (N, D) feature vectors (SS, CHI용)
    topics: topic word lists (topic coherence, diversity용)
    texts: raw docs (coherence metric 계산용)
    """
    # Hungarian 매칭 (외부 함수 필요)
    mapping, y_pred_aligned = hungarian_match(labels_true, labels_pred)

    # External metrics
    acc = accuracy_score(labels_true, y_pred_aligned)
    recalls = recall_score(labels_true, y_pred_aligned, average=None, zero_division=0)
    f1_w = f1_score(labels_true, y_pred_aligned, average="weighted")
    ari = adjusted_rand_score(labels_true, labels_pred)
    hs  = homogeneity_score(labels_true, labels_pred)
    report = classification_report(labels_true, y_pred_aligned, zero_division=0, output_dict=True)

    # Internal clustering metrics
    ss, chi = None, None
    if X is not None and len(set(labels_pred)) > 1:
        try:
            ss  = silhouette_score(X, labels_pred)
            chi = calinski_harabasz_score(X, labels_pred)
        except Exception as e:
            print("[warn] Internal metrics failed:", e)

    # Topic modeling metrics
    npmi, td = None, None
    if topics is not None:
        npmi = compute_npmi(topics, texts)
        td   = compute_td(topics)

    return {
        "mapping": mapping,
        "accuracy": acc,
        "recall_per_class": recalls.tolist(),
        "f1_weighted": f1_w,
        "ari": ari,
        "homogeneity": hs,
        "classification_report": report,
        "y_pred_aligned": y_pred_aligned,
        "silhouette": ss,
        "calinski_harabasz": chi,
        "topic_coherence_npmi": npmi,
        "topic_diversity": td,
    }

metrics = evaluate_with_hungarian(y_true, y_pred, X=X, topics=topics, texts=texts)

print("=== External Metrics (라벨 필요) ===")
print(f"Accuracy         : {metrics['accuracy']:.4f}")
print(f"F1-weighted     : {metrics['f1_weighted']:.4f}")
print(f"ARI             : {metrics['ari']:.4f}")
print(f"Homogeneity     : {metrics['homogeneity']:.4f}")

print("\n=== Internal Metrics (라벨 없이 군집 품질) ===")
print(f"Silhouette Score: {metrics['silhouette']}")
print(f"Calinski-Harabasz Index: {metrics['calinski_harabasz']}")

print("\n=== Topic Modeling Metrics ===")
print(f"NPMI Coherence  : {metrics['topic_coherence_npmi']}")
print(f"Topic Diversity : {metrics['topic_diversity']}")
# label 이름이 있으면 더 깔끔
label_names = {i: f"class_{i}" for i in range(20)}  # 필요 시 실제 이름 매핑
metrics = evaluate_with_hungarian(y_true, y_pred, X=X, topics=topics, texts=texts)
class_hungarian_metrics(metrics, label_names=label_names, save_prefix=None)
### BERTopic Metric
# 1) 토큰 코퍼스 만들기 (영어 기준: 소문자/알파벳, 불용어/한 글자 제거)
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.evaluation_metrics.coherence_metrics import Coherence

STOP = set(ENGLISH_STOP_WORDS) | {
    # 수축형/노이즈 보완
    "don","ve","ll","re","im","ms","nd","uw","ww"
}
_token_pat = re.compile(r"[A-Za-z]+")

def corpus_to_tokens(texts, min_len=2):
    corpus_tokens = []
    for t in texts:
        toks = [w.lower() for w in _token_pat.findall(str(t) or "")]
        toks = [w for w in toks if len(w) >= min_len and w not in STOP]
        corpus_tokens.append(toks)
    return corpus_tokens

corpus_tokens = corpus_to_tokens(texts)  # <-- 주신 numpy array 그대로 전달
def evaluate(self, output_tm):
        """Using metrics and output of the topic model, evaluate the topic model"""
        if self.timestamps:
            results = {str(timestamp): {} for timestamp, _ in output_tm.items()}
            for timestamp, topics in output_tm.items():
                self.metrics = self.get_metrics()
                for scorers, _ in self.metrics:
                    for scorer, name in scorers:
                        score = scorer.score(topics)
                        results[str(timestamp)][name] = float(score)

        else:
            # Calculate results
            results = {}
            for scorers, _ in self.metrics:
                for scorer, name in scorers:
                    score = scorer.score(output_tm)
                    results[name] = float(score)

            # Print results
            if self.verbose:
                print("Results")
                print("============")
                for metric, score in results.items():
                    print(f"{metric}: {str(score)}")
                print(" ")

        return results

def get_metrics(self):
        """Prepare evaluation measures using OCTIS"""
        npmi = Coherence(texts=self.data.get_corpus(), topk=self.topk, measure="c_npmi")
        topic_diversity = TopicDiversity(topk=self.topk)

        # Define methods
        coherence = [(npmi, "npmi")]
        diversity = [(topic_diversity, "diversity")]
        metrics = [(coherence, "Coherence"), (diversity, "Diversity")]

        return metrics
# 1) 토큰화 (이미 있는 함수)
corpus_tokens = corpus_to_tokens(texts)

# 2) 실행부 전용 정제 (빈 문서 제거 + vocab 필터 + 길이>=2 토픽만 유지)
def sanitize_for_coherence_exec_side(corpus_tokens, topics, max_topk=10):
    nonempty_docs = [doc for doc in corpus_tokens if len(doc) > 0]
    vocab = set(w for doc in nonempty_docs for w in doc)

    topics_clean = []
    for t in topics:
        tt = [w for w in t if w in vocab]
        if len(tt) >= 2:
            topics_clean.append(tt)

    if not nonempty_docs:
        raise ValueError("All documents became empty after tokenization.")
    if not topics_clean:
        raise ValueError("All topics became <2 words after filtering.")

    # 핵심: 모든 토픽 길이를 만족하는 safe topk 산출
    min_len = min(len(t) for t in topics_clean)
    effective_topk = min(max_topk, min_len)

    # 각 토픽을 effective_topk로 슬라이스 (OCTIS가 topk 상위만 사용)
    topics_trimmed = [t[:effective_topk] for t in topics_clean]

    print(f"[info] docs={len(nonempty_docs)} | topics_in={len(topics)} "
          f"| topics_used={len(topics_trimmed)} | min_topic_len={min_len} "
          f"| topk={effective_topk}")

    return nonempty_docs, topics_trimmed, effective_topk

corpus_tokens_clean, topics_clean, effective_topk = sanitize_for_coherence_exec_side(
    corpus_tokens, topics, max_topk=10
)

# 3) Dummy 객체는 수정 없이, 다만 topk를 safe 값으로 전달
class Dummy:
    def __init__(self, corpus_tokens, verbose=True, topk=10):
        self.data = self
        self._corpus = corpus_tokens
        self.verbose = verbose
        self.topk = topk
        self.timestamps = None
        self.metrics = self.get_metrics()

    def get_corpus(self):
        return self._corpus

    evaluate = evaluate
    get_metrics = get_metrics

dummy = Dummy(corpus_tokens_clean, verbose=True, topk=effective_topk)

# 4) OCTIS는 dict 입력 필요
results = dummy.evaluate({"topics": topics_clean})
print("Final results:", results)